# Dictionary based time series classification in sktime

Dictionary based approaches adapt the bag of words model commonly used in signal processing, computer vision and audio processing for time series classification.
Dictionary based classifiers have the same broad structure.
A sliding window of length $w$ is run across a series.
For each window, the real valued series of length $w$ is converted through approximation and discretisation processes into a symbolic string of length $l$, which consists of $\alpha$ possible letters.
The occurrence in a series of each 'word' from the dictionary defined by $l$ and $\alpha$ is counted, and once the sliding window has completed the series is transformed into a histogram.
Classification is based on the histograms of the words extracted from the series, rather than the raw data.

Currently 3 dictionary based classifiers and implemented in sktime, both making use of the Symbolic Fourier Approximation (SFA)\[1\] transform to discretise into words.
These are the Bag of SFA Symbols (BOSS)\[2\], with options for improved efficiency and usability via contracting (cBOSS)\[3\] and the Temporal Dictionary Ensemble (TDE)\[4\]

In this notebook, we will demonstrate how to use BOSS, cBOSS and TDE on the ItalyPowerDemand dataset. Both algorithms are currently only compatible with univariate time series datasets.

#### References:

\[1\] Schäfer, P., & Högqvist, M. (2012). SFA: a symbolic fourier approximation and index for similarity search in high dimensional datasets. In Proceedings of the 15th International Conference on Extending Database Technology (pp. 516-527).

\[2\] Schäfer, P. (2015). The BOSS is concerned with time series classification in the presence of noise. Data Mining and Knowledge Discovery, 29(6), 1505-1530.

\[3\] Middlehurst, M., Vickers, W., & Bagnall, A. (2019). Scalable dictionary classifiers for time series classification. In International Conference on Intelligent Data Engineering and Automated Learning (pp. 11-19). Springer, Cham.

\[4\] Middlehurst, M., Large, J., Cawley, G., & Bagnall, A. (2020). The Temporal Dictionary Ensemble (TDE) Classifier for Time Series Classification. In The European Conference on Machine Learning and Principles and Practice of Knowledge Discovery in Databases.

\[5\] Large, J., Bagnall, A., Malinowski, S., & Tavenard, R. (2019). On time series classification with dictionary-based classifiers. Intelligent Data Analysis, 23(5), 1073-1089.

\[6\] Schäfer, P., & Leser, U. (2017). Fast and accurate time series classification with weasel. In Proceedings of the 2017 ACM on Conference on Information and Knowledge Management (pp. 637-646).

## 1. Imports

In [1]:
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.dictionary_based import TemporalDictionaryEnsemble

from sklearn import metrics
from sktime.datasets import load_italy_power_demand

## 2. Load data

In [2]:
X_train, y_train = load_italy_power_demand(split='train', return_X_y=True)
X_test, y_test = load_italy_power_demand(split='test', return_X_y=True)
X_test = X_test[:50]
y_test = y_test[:50]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(67, 1) (67,) (50, 1) (50,)


## 3. Bag of SFA Symbols (BOSS)

BOSS is an ensemble of individual BOSS classifiers making use of the SFA transform.
The classifier performs grid-search through a large number of individual classifiers for parameters $l$, $\alpha$, $w$ and $p$ (normalise each window).
Of the classifiers searched only those within 92\% accuracy of the best classifier are retained.
Individual BOSS classifiers use a non-symmetric distance function, BOSS distance, in conjunction with a nearest neighbour classifier.

As tuning is handled inside the classifier, BOSS has very little parameters to be altered and generally should be run using default settings.

In [3]:
boss = BOSSEnsemble()
boss.fit(X_train, y_train)

boss_preds = boss.predict(X_test)
print("BOSS Accuracy: " + str(metrics.accuracy_score(y_test, boss_preds)))

BOSS Accuracy: 0.86


## 4. Contractable BOSS (cBOSS)

cBOSS significantly speeds up BOSS with no signficant difference in accuracy by improving how the ensemble is formed.
cBOSS utilises a filtered random selection of parameters to find its ensemble members.
Each ensemble member is built on a 70% subsample of the train data, using random sampling without replacement.
An exponential weighting scheme for the predictions of the base classifiers is introduced.

A new parameter for the number of parameters samples $k$ is introduced. of which the top $s$ (max ensemble size) with the highest accuracy are kept for the final ensemble.
The $k$ parameter is replaceable with a time limit $t$ through contracting.

In [4]:
# Recommended non-contract cBOSS parameters
cboss = BOSSEnsemble(randomised_ensemble=True,
                    n_parameter_samples=250,
                    max_ensemble_size=50)

# cBOSS with a 5 minute build time contract
#cboss = BOSSEnsemble(randomised_ensemble=True,
#                    time_limit=5,
#                    max_ensemble_size=50)

cboss.fit(X_train, y_train)

cboss_preds = cboss.predict(X_test)
print("cBOSS Accuracy: " + str(metrics.accuracy_score(y_test, cboss_preds)))

cBOSS Accuracy: 0.9


## 5. Temporal Dictionary Ensemble (TDE)

TDE aggregates the best compoents of 3 classifiers extending from the original BOSS algorithm. The ensemble structure and improvements of cBOSS are used; Spatial pyramids are introduced from Spatial BOSS (S-BOSS)\[5\]; From Word Extraction for Time Series Classification (WEASEL)\[6\] bigrams and Information Gain Binning (IGB), a replacement for the multiple coefficient binning (MCB) used by SFA, are included.
Two new parameters are included in the ensemble parameter search, the number of spatial pyramid levels $h$ and whether to use IGB or MCB $b$.
A Gaussian processes regressor is used to select new parameter sets to evaluate for the ensemble, predicting the accuracy of a set of parameter values using past classifier performances.

Inheriting the cBOSS ensemble structure, the number of parameter samples $k$, time limit $t$ and max ensemble size $s$ remain as parameters to be set accounting for memory and time requirements.

In [5]:
# Recommended non-contract TDE parameters
tde = TemporalDictionaryEnsemble(n_parameter_samples=250,
                                 max_ensemble_size=100,
                                 randomly_selected_params=50)

# TDE with a 5 minute build time contract
#tde = TemporalDictionaryEnsemble(time_limit=5,
#                                 max_ensemble_size=100,
#                                 randomly_selected_params=50)

tde.fit(X_train, y_train)

tde_preds = tde.predict(X_test)
print("TDE Accuracy: " + str(metrics.accuracy_score(y_test, tde_preds)))

TDE Accuracy: 0.98
